In [25]:
import pymongo
import quandl
import pandas as pd
import json
import numpy as np
from datetime import datetime,timedelta

quandl.ApiConfig.api_key = ''

In [3]:
mongo_connect_string = 'mongodb://TwitterIPO?authSource=TwitterIPO'
client = pymongo.MongoClient(mongo_connect_string)
db = client.TwitterIPO
RawTweets = db.RawTweets
PriceData = db.PriceData
ProcessedTweets = db.ProcessedTweets

In [6]:
ipo_list = pd.read_excel('IPO-Last100.xlsx')

In [7]:
ipo_list = ipo_list[['Symbol','Offer Date']]

In [8]:
ipo_list['End Date'] = ipo_list['Offer Date'] + timedelta(days=30)

In [9]:
ipo_list.loc[ipo_list['End Date'] > datetime(2017,3,25), 'End Date'] = datetime(2017,3,25)

In [43]:
################################################################
### For stocks in Dataframe ipo_list - Grab the data from Quandl
### Insert into MongoDB
###
###
###
###
###


for index,row in ipo_list.iterrows():
    
    # Set Symbol, Start and End Dates
    symbol = str(row['Symbol'])
    start = row['Offer Date'].date()
    end = row['End Date'].date()
    
    # Send Request to Quandl
    data = quandl.get('EOD/' + symbol, start_date=start, end_date=end)
    
    # Optimize Quandl's output (which is a df) - Play around with Indexes etc for inserting into MongoDB
    data['Name'] = symbol
    
    
    # Generate Unique ID
    data.reset_index(inplace=True)
    data.index.astype('str')
    data['UID'] = data['Name'].apply(str) + data.index.astype('str')   
    #data = data.set_index(['UID'])
    
    
    
    # Calculate Return and assign Class
    data.sort_values('Date')
    data['return'] = data.Adj_Close.pct_change()
    data['return'].fillna((data['Adj_Close'] / data['Adj_Open']) - 1,inplace=True)
    data['direction'] = np.where(data['return'] >= 0, 'Up','Down')
    data['class_num_label'] = data.direction.map({'Down' : 0, 'Up' : 1})
    
    
    #test_result.append(data)
    
    # Convert to JSON for MongoDB
    
    records = json.loads(data.to_json(orient='index',date_format='iso')).values()
    
    # Insert into MongoDB
    
    PriceData.insert_many(records)

C:\Users\Saifi\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\computation\expressions.py:62: FutureWarning: using '+' to provide set union with Indexes is deprecated, use '|' or .union()
  return op(a, b)
